In [12]:
# Import
import numpy as np
if not hasattr(np, "product"):         
    np.product = np.prod

import pandas as pd
from pyprocessmacro import Process

df = (
    pd.read_excel(r"File_Path")
      .rename(columns={
          "AIGL"   : "X",    # AI‑Generated Label
          "Ave_PA" : "M1",   # Perception on Authenticity
          "Ave_CT" : "M2",   # Consumer Trust
          "PT"     : "W1",   # Platform Type         → moderator w
          "FE"     : "W2"    # Facial Expression     → moderator z
      })
)

# encode categoricals
for col in ["X", "W1", "W2"]:
    if df[col].dtype == "object":
        df[col] = pd.Categorical(df[col]).codes

mod9 = Process(
    data   = df,
    x      = "X",      # AI‑Generated Label
    y      = "M2",     # Consumer Trust (outcome)
    m      = ["M1"],   # Perception on Authenticity
    w      = "W1",     # Platform Type  (moderates X→M2)
    z      = "W2",     # Facial Expression (moderates M1→M2)
    model  = 9,
    n_boot = 5000,
    seed   = 42
)

mod9.summary()

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 9

Variables:
    Cons = Cons
    x = X
    y = M2
    m1 = M1
    w = W1
    z = W2

Sample size:
205

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = M2 
OLS Regression Summary

     R²  Adj. R²    MSE        F  df1  df2  p-value
 0.6974   0.6929 0.2399 232.8279    2  202   0.0000

Coefficients

      coeff     se       t      p    LLCI   ULCI
Cons 0.5322 0.2466  2.1581 0.0321  0.0488 1.0155
X    0.0546 0.0763  0.7157 0.4750 -0.0949 0.2042
M1   0.9010 0.0474 19.0246 0.0000  0.8082 0.9938

-------------------------------------------------------------------------

Outcome = M1 
OLS Regression Summary

     R²  Adj. R²  

C:\Users\Dian Napitupulu\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyprocessmacro\models.py:752: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  llci, ulci = bias_corrected_ci(e, be, conf)
C:\Users\Dian Napitupulu\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyprocessmacro\models.py:1188: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return df.apply(pd.to_numeric, args=["ignore"])
C:\Users\Dian Napitupulu\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyprocessmacro\models.py:752: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest',

In [14]:
# Import
import numpy as np
if not hasattr(np, "product"):     
    np.product = np.prod

import pandas as pd
import statsmodels.formula.api as smf
from pyprocessmacro import Process

df = (
    pd.read_excel(r"File_Path")
      .rename(columns={
          "AIGL"   : "X",   # AI‑Generated Label
          "Ave_PA" : "M1",  # Perception on Authenticity
          "Ave_CT" : "M2",  # Consumer Trust
          "Ave_PI" : "Y",   # Purchase Intention
          "PT"     : "W1",  # Platform Type          – moderator w
          "FE"     : "W2"   # Facial Expression      – moderator z
      })
)

# encode categoricals
for col in ["X", "W1", "W2"]:
    if df[col].dtype == "object":
        df[col] = pd.Categorical(df[col]).codes

# ensure numeric Y
df["Y"] = pd.to_numeric(df["Y"], errors="coerce")

# PROCESS Model 9
mod9 = Process(
    data   = df,
    x      = "X",
    y      = "M2",
    m      = ["M1"],
    w      = "W1",        # moderates X→M2
    z      = "W2",        # moderates M1→M2   (Model 9 requires this)
    model  = 9,
    n_boot = 5000,
    seed   = 42
)

print("\n=== PROCESS Model 9 summary ===")
mod9.summary()        

# Consumer Trust → Purchase Intention
y_mod = smf.ols(
    "Y ~ M2 + X + M1 + W1 + W2",   # kontrol X, M1, & kedua moderator
    data=df
).fit(cov_type="HC3")

print("\n=== Regressi Purchase Intention ===")
print(y_mod.summary().tables[1])   # Coefficient table
b4 = y_mod.params["M2"]            # M2 → Y

# Bootstrap chain indirect effects   X→M1→M2→Y
def bootstrap_chain(data, n_boot=5000, seed=42,
                    w1_val=None, w2_val=None):
    """
    Bootstraps indirect effect X → M1 → M2 → Y
    at specified W1 and W2 levels (defaults = sample means).
    """
    rng   = np.random.default_rng(seed)
    n     = len(data)
    dist  = []

    # default: mean moderator
    w1_val = data["W1"].mean() if w1_val is None else w1_val
    w2_val = data["W2"].mean() if w2_val is None else w2_val

    for _ in range(n_boot):
        samp = data.sample(n, replace=True, random_state=rng)

        m9 = Process(
            data   = samp,
            x      = "X",
            y      = "M2",
            m      = ["M1"],
            w      = "W1",
            z      = "W2",
            model  = 9,
            n_boot = 0
        )

        # ---- Model 9 ----
        a1      = m9.params_path["X→M1"]              # X → M1
        d21     = m9.params_path["M1→M2"]             # M1 → M2
        d21_z   = m9.params_path.get("M1:W2→M2", 0)   # interaksi M1×W2

        # conditional M1 → M2 given W2
        cond_d21 = d21 + d21_z * w2_val

        # ---- M2 → Y in bootstrap sample ----
        b4_boot = smf.ols("Y ~ M2 + X + M1 + W1 + W2",
                          data=samp).fit().params["M2"]

        dist.append(a1 * cond_d21 * b4_boot)

    return np.array(dist)

chain_dist = bootstrap_chain(df)
chain_mean = chain_dist.mean()
ci_low, ci_high = np.percentile(chain_dist, [2.5, 97.5])

print("\n=== Indirect berantai  X → M1 → M2 → Y ===")
print(f"Boot‑mean = {chain_mean:.4f}")
print(f"95 % CI    = [{ci_low:.4f}, {ci_high:.4f}]")


C:\Users\Dian Napitupulu\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyprocessmacro\models.py:752: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  llci, ulci = bias_corrected_ci(e, be, conf)
C:\Users\Dian Napitupulu\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyprocessmacro\models.py:1188: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return df.apply(pd.to_numeric, args=["ignore"])
C:\Users\Dian Napitupulu\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyprocessmacro\models.py:752: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest',

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 9

Variables:
    Cons = Cons
    x = X
    y = M2
    m1 = M1
    w = W1
    z = W2

Sample size:
205

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

=== PROCESS Model 9 summary ===

***************************** OUTCOME MODELS ****************************

Outcome = M2 
OLS Regression Summary

     R²  Adj. R²    MSE        F  df1  df2  p-value
 0.6974   0.6929 0.2399 232.8279    2  202   0.0000

Coefficients

      coeff     se       t      p    LLCI   ULCI
Cons 0.5322 0.2466  2.1581 0.0321  0.0488 1.0155
X    0.0546 0.0763  0.7157 0.4750 -0.0949 0.2042
M1   0.9010 0.0474 19.0246 0.0000  0.8082 0.9938

-------------------------------------------------------------------------

Outcome = M1 
OLS Regre

C:\Users\Dian Napitupulu\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyprocessmacro\models.py:752: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  llci, ulci = bias_corrected_ci(e, be, conf)


AttributeError: 'Process' object has no attribute 'params_path'